# Interacción con dos piedras
Modelando la Interacción entre Dos Piedras en un Tablero de Go con QUBO
En este código, se utiliza un modelo QUBO (Quadratic Unconstrained Binary Optimization) para representar la interacción entre dos piedras en un tablero de Go. Aquí están los puntos clave:

Definición del Solver: Se utiliza el DWaveSampler de D-Wave con el tipo de solver Advantage2_prototype2.3 y un token específico para acceder al QPU (Quantum Processing Unit).
Coeficientes del Hamiltoniano QUBO: Se define un diccionario Q que contiene los coeficientes del Hamiltoniano QUBO. Estos coeficientes representan las interacciones entre las variables binarias que modelan las piedras en el tablero.
Ejecución del Problema: El problema QUBO se ejecuta en el QPU de D-Wave utilizando el sampler.sample_qubo con un número específico de lecturas (num_reads).
Procesamiento de Resultados: Los resultados obtenidos se procesan para convertir las variables binarias a los estados originales de las piedras (S1 y S2) y se comparan con una tabla original para verificar la precisión.

In [1]:
import numpy as np
import pandas as pd
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod
import dwave.inspector as inspector

# Definir el solver
solver_type = 'Advantage2_prototype2.3'
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"
solver = DWaveSampler(solver=solver_type, token=dwave_token)
sampler = EmbeddingComposite(solver)

# Definir los coeficientes del Hamiltoniano QUBO
P = 10  # Término de penalización

# Construir el diccionario QUBO
Q = {
    ('x01', 'x01'): 1,
    ('x02', 'x02'): -1,
    ('x11', 'x11'): 2,
    ('x12', 'x12'): -2,
    ('x01', 'x11'): -2,
    ('x01', 'x12'): 0,
    ('x02', 'x11'): 0,
    ('x02', 'x12'): 2,
    ('x01', 'x02'): P,
    ('x11', 'x12'): P
}

# Ejecutar el problema en el QPU de D-Wave
num_reads = 1000
response = sampler.sample_qubo(Q, num_reads=num_reads, return_embedding=True)

# Usar el inspector de D-Wave
inspector.show(response)

# Procesar los resultados
results = []
for sample, energy, num_occurrences in response.data(['sample', 'energy', 'num_occurrences']):
    x01, x02, x11, x12 = sample['x01'], sample['x02'], sample['x11'], sample['x12']
    
    # Convertir las variables binarias a los estados originales de S1 y S2
    S1 = 1 if x01 == 1 and x02 == 0 else (-1 if x01 == 0 and x02 == 1 else 0)
    S2 = 1 if x11 == 1 and x12 == 0 else (-1 if x11 == 0 and x12 == 1 else 0)
    
    results.append({
        'Estado Original': f"({S1}, {S2})",
        'x01': x01,
        'x02': x02,
        'x11': x11,
        'x12': x12,
        'Energía QUBO': energy,
        'Ocurrencias': num_occurrences
    })

# Crear un DataFrame con los resultados
df_results = pd.DataFrame(results)
print("Resultados obtenidos:")
print(df_results)

# Comparar con la tabla original
original_data = [
    ((1, 1), 1, 0, 1, 0, 1.0),
    ((1, -1), 1, 0, 0, 1, -1.0),
    ((-1, 1), 0, 1, 1, 0, 1.0),
    ((-1, -1), 0, 1, 0, 1, -1.0),
    ((1, 0), 1, 0, 0, 0, 1.0),
    ((-1, 0), 0, 1, 0, 0, -1.0),
    ((0, 1), 0, 0, 1, 0, 2.0),
    ((0, -1), 0, 0, 0, 1, -2.0),
    ((0, 0), 0, 0, 0, 0, 0.0)
]

df_original = pd.DataFrame(original_data, columns=['Estado Original', 'x01', 'x02', 'x11', 'x12', 'Energía Esperada'])

# Comparar los resultados obtenidos con los esperados
comparison = df_results.merge(df_original, on=['Estado Original', 'x01', 'x02', 'x11', 'x12'], how='outer')
comparison['Diferencia'] = comparison['Energía QUBO'] - comparison['Energía Esperada']
print("\nComparación con la tabla original:")
print(comparison[['Estado Original', 'x01', 'x02', 'x11', 'x12', 'Energía QUBO', 'Energía Esperada', 'Diferencia']])

print("\nResumen de estados encontrados:")
print(df_results['Estado Original'].value_counts())

Serving Inspector on http://127.0.0.1:18001/?problemId=fa2e20e6-8284-40de-8a2a-5987739c21b1

Resultados obtenidos:
  Estado Original  x01  x02  x11  x12  Energía QUBO  Ocurrencias
0         (0, -1)    0    0    0    1          -2.0          867
1        (-1, -1)    0    1    0    1          -1.0           53
2         (-1, 0)    0    1    0    0          -1.0           47
3         (1, -1)    1    0    0    1          -1.0           32
4          (0, 0)    0    0    0    0           0.0            1

Comparación con la tabla original:
   Estado Original  x01  x02  x11  x12  Energía QUBO  Energía Esperada  \
0          (0, -1)    0    0    0    1          -2.0               NaN   
1         (-1, -1)    0    1    0    1          -1.0               NaN   
2          (-1, 0)    0    1    0    0          -1.0               NaN   
3          (1, -1)    1    0    0    1          -1.0               NaN   
4           (0, 0)    0    0    0    0           0.0               NaN   
5           (1, 1)    1    0    1    0           NaN               1.0   
6          (1, -1)    1    0    0 

# Interacción con n piedras
Este script utiliza un modelo QUBO para representar la interacción entre varias piedras en un tablero de Go. Aquí están los puntos clave:

Inicialización de variables y configuraciones: Se establece el tipo de solver y el token de D-Wave para acceder al ordenador cuántico. También se define un término de penalización P.
Definición del número de piedras (n): Se define el número de piedras que se van a modelar. En este caso, se modelan n = 5 piedras.
Generación de variables QUBO: Se genera un diccionario Q que representa los coeficientes del Hamiltoniano QUBO. Cada piedra está representada por dos variables binarias (x{i}0 y x{i}1), y los coeficientes se asignan de tal manera que modelan la interacción entre las piedras. Además, se añade un término de penalización para asegurar que solo una variable por piedra sea 1.
Ejecución del problema en el QPU de D-Wave: Se ejecuta el problema QUBO en el ordenador cuántico D-Wave y se obtiene una respuesta. Se ajusta la fuerza de las cadenas, que puede necesitar ser optimizada.
Visualización de los resultados con el inspector: Se utiliza el inspector de D-Wave para visualizar la respuesta.
Procesamiento de los resultados: Se procesan los resultados, convirtiendo las variables binarias a los estados originales y almacenando los resultados en un DataFrame de pandas.

In [3]:
import numpy as np
import pandas as pd
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod
import dwave.inspector as inspector

# 1. Inicializar variables y configuraciones
solver_type = 'Advantage2_prototype2.3'
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"
solver = DWaveSampler(solver=solver_type, token=dwave_token)
sampler = EmbeddingComposite(solver)
P = 10  # Término de penalización

# 2. Definir el número de piedras (n)
n = 5  # Ejemplo para 3 piedras, ajustar según sea necesario

# 3. Generar variables QUBO
Q = {}
for i in range(n):
    for j in range(2):  # Dos variables por piedra
        Q[(f'x{i}{j}', f'x{i}{j}')] = (-1)**j * (j+1)  # Asignar coeficientes
        if j == 0:  # Término de penalización para asegurar que solo una variable por piedra sea 1
            for k in range(n):
                if k != i:
                    Q[(f'x{i}0', f'x{k}0')] = P

# 4. Ejecutar el problema en el QPU de D-Wave
num_reads = 1000
chain_strength = 10*P  # Ajustar la fuerza de las cadenas, puede necesitar ser optimizado
response = sampler.sample_qubo(Q, num_reads=num_reads, chain_strength=chain_strength, return_embedding=True)

# Visualizar los resultados con el inspector
inspector.show(response)

# 5. Procesar los resultados
results = []
for sample, energy, num_occurrences in response.data(['sample', 'energy', 'num_occurrences']):
    state = tuple(sample[f'x{i}{j}'] for i in range(n) for j in range(2))
    results.append({'Estado': state, 'Energía QUBO': energy, 'Ocurrencias': num_occurrences})

# 6. Mostrar los resultados
df_results = pd.DataFrame(results)
print("Resultados obtenidos:")
print(df_results)



Serving Inspector on http://127.0.0.1:18001/?problemId=ceb45290-10fc-41f8-aa87-858adf6b3e51

Resultados obtenidos:
                             Estado  Energía QUBO  Ocurrencias
0    (0, 1, 0, 1, 0, 1, 0, 1, 0, 1)         -10.0           11
1    (0, 1, 1, 1, 0, 1, 0, 1, 0, 1)          -9.0           10
2    (0, 1, 0, 1, 1, 1, 0, 1, 0, 1)          -9.0            8
3    (1, 1, 0, 1, 0, 1, 0, 1, 0, 1)          -9.0            9
4    (0, 1, 0, 1, 0, 1, 0, 1, 1, 1)          -9.0           11
..                              ...           ...          ...
269  (0, 0, 1, 0, 0, 0, 1, 1, 0, 0)          20.0            1
270  (0, 1, 0, 0, 1, 0, 0, 0, 1, 0)          20.0            1
271  (0, 0, 1, 0, 0, 1, 1, 0, 0, 0)          20.0            1
272  (1, 0, 0, 0, 0, 0, 1, 0, 0, 0)          22.0            1
273  (0, 1, 1, 1, 0, 0, 1, 0, 1, 1)          57.0            1

[274 rows x 3 columns]
